# SQLite

In [15]:
import pandas as pd
import sqlite3 as sql

#since we are looking at poems, we'll bump up the width of our columns
pd.set_option("display.max_colwidth", 250)

### Creating a connection to the poetry_kids.db database
 - tell python what database to connect to (`sql.connect( )`)
 - create a cursor to allow passing queries directly to the database

In [16]:
conn = sql.connect('../data/poetry_kids.db')
cur = conn.cursor()

### Checking what tables exist within a database
- the cursor executes the SQL query passed to `.execute( )`
- the results of your query are loaded to the cursor
- the `fetchall()` method returns those results so you can store them in a variable

In [17]:
# Execute tells the cursor to run the query provided

cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

# fetchall() allows us to see everything returned by our query, which we store in our 'available_tables' variable

available_tables = (cur.fetchall())

available_tables

[('author',),
 ('emotion',),
 ('gender',),
 ('grade',),
 ('poem',),
 ('poem_emotion',)]

Let's compare this to the **entity relationship diagram** (ERD) for this database.

![ERD](assets/PoetryKids_erd.png)

## Using Pandas to place results into a DataFrame

We can SQL queries and convert the result to a _pandas_ DataFrame by using the `read_sql` method.

When building SQL queries, it may be useful to write the query independently and save to variable. Multi-line queries can use triple quotes (''' ''') to keep it together as one string.

For example, if we want to find the very short poems, we can use the following code. 

In [18]:
query = '''
SELECT text
FROM poem
WHERE char_count < 20;
'''

pd.read_sql(query, conn)

,text
0,hello friend
1,bye
2,i like chicken
3,"star, star,"
4,lol life rules
5,ghosts use magic
6,i love u
7,dragons are cool
8,may you bee ears
9,the time is gone


Let's now practice writing queries. Remember the process we can use:

    a. decide what tables you need   
    b. (if you need to use more than one table) decide how to connect your tables   
    c. decide which columns to select  
    d. decide what (if any) filters you want to apply in your `WHERE` clause

First, let's find the text of all poems with the title "cats".

In [19]:
query = '''
SELECT title
FROM poem
WHERE title = 'cats'
ORDER BY title;
'''

pd.read_sql(query, conn)

,title
0,cats
1,cats
2,cats
3,cats
4,cats
5,cats
6,cats
7,cats
8,cats
9,cats


Now, let's see find the emotion_id associated with these poems. Keep the text of the poem, the poem_id, the emotion_id, and the intensity_percent.

Sort by the poem_id as the same poem can appear multiple times.

In [20]:
query = '''
SELECT po.text,pe.poem_id,pe.emotion_id,pe.intensity_percent
FROM poem as po
LEFT JOIN poem_emotion as pe
ON po.id = pe.poem_id
WHERE po.title = 'cats'
ORDER BY pe.poem_id;
'''

pd.read_sql(query, conn)

,text,poem_id,emotion_id,intensity_percent
0,i like cats look at those hats but never mind that i like cats cats cats cats,NaN,NaN,NaN
1,cats are cute cats are cute i llike cats what about you,NaN,NaN,NaN
2,cats are really cute they cause a lot of trouble who cares i dont mind,NaN,NaN,NaN
3,cats are meowing and laying under the oak playing with a mouse,NaN,NaN,NaN
4,i love cats when they make sounds like this meow meow cat are raelly cute when they meow,1948.0,4.0,83.0
5,cats aer fun cats are gerat,4508.0,4.0,79.0
6,"furry ones, black ones, stript ones too every cat is different and so are you",4866.0,3.0,28.0
7,cute awesome treats special,10232.0,4.0,57.0
8,i am very fury i climb trees when i get scard and i go meow,14527.0,1.0,92.0
9,i am very fury i climb trees when i get scard and i go meow,14527.0,2.0,67.0


Finally, let's modify the above query to show the emotion name instead of the emotion_id. This will require joining with another table.

In [24]:
query = '''
SELECT text,poem_id,emotion.name,intensity_percent
FROM poem_emotion
LEFT JOIN poem
ON poem_id = poem.id
LEFT JOIN emotion
ON poem_emotion.emotion_id = emotion.id
WHERE title ='cats'
ORDER BY poem_id;
'''


In [22]:
query = '''
SELECT po.text,pe.poem_id,emotion.name,pe.intensity_percent
FROM poem as po
LEFT JOIN poem_emotion as pe
ON po.id = pe.poem_id
LEFT JOIN emotion AS e
ON emotion_id = emotion.id
WHERE po.title = 'cats'
ORDER BY pe.poem_id;
'''


pd.read_sql(query, conn)

DatabaseError: Execution failed on sql '
SELECT po.text,pe.poem_id,emotion.name,pe.intensity_percent
FROM poem as po
LEFT JOIN poem_emotion as pe
ON po.id = pe.poem_id
LEFT JOIN emotion AS e
ON emotion_id = emotion.id
WHERE po.title = 'cats'
ORDER BY pe.poem_id;
': no such column: emotion.name

## Best Practice - Close connections after use to prevent the tables from being locked

In [12]:
cur.close()
conn.close()